In [10]:
%load_ext autoreload
%autoreload 2

import os
import json
import itertools
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import collections

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio

import tss
import sklearn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from sklearn.datasets import fetch_openml

co2 = fetch_openml(data_id=41187, as_frame=True)
co2.frame.head()

,year,month,day,weight,flag,station,co2
0,1958.0,3.0,29.0,4.0,0.0,MLO,316.1
1,1958.0,4.0,5.0,6.0,0.0,MLO,317.3
2,1958.0,4.0,12.0,4.0,0.0,MLO,317.6
3,1958.0,4.0,19.0,6.0,0.0,MLO,317.5
4,1958.0,4.0,26.0,2.0,0.0,MLO,316.4


In [12]:
import pandas as pd

co2_data = co2.frame
co2_data["date"] = pd.to_datetime(co2_data[["year", "month", "day"]])
co2_data = co2_data[["date", "co2"]].set_index("date")
co2_data.head()

,co2
date,
1958-03-29,316.1
1958-04-05,317.3
1958-04-12,317.6
1958-04-19,317.5
1958-04-26,316.4


In [42]:
def get_mauna_lua():
    co2 = sklearn.datasets.fetch_openml(data_id=41187, as_frame=True)
    co2_data = co2.frame
    co2_data["date"] = pd.to_datetime(co2_data[["year", "month", "day"]])
    co2_data = co2_data[["date", "co2"]].set_index("date")
    X = (co2_data.index.year + co2_data.index.month / 12).to_numpy().reshape(-1, 1)
    y = co2_data["co2"].to_numpy()
    return X, y


def split_dataset(X, y, step=10):
    assert X.shape[0] == y.shape[0]
    for start in range(0, X.shape[0], step):
        cur_x, cur_y = X[start:start+step], y[start:start+step]
        yield np.pad(cur_x, [(0, step - cur_x.shape[0]), (0, 0)]),\
              np.pad(cur_y, step - cur_y.shape[0])

In [43]:
X, y = get_mauna_lua()

In [44]:
ds = list(split_dataset(X, y))

In [45]:
X = np.array([el[0] for el in ds])

In [46]:
X.shape

(223, 10, 1)

In [57]:
def load_arff(path: str) -> pd.DataFrame:
    data = arff.loadarff(path)
    return pd.DataFrame(data[0])

In [71]:
df = load_arff("../data/EEG Eye State.arff")
X = df.drop("eyeDetection", axis=1).to_numpy()
y = df["eyeDetection"].astype(np.int).to_numpy()


In [75]:
def get_eeg():
    df = load_arff("../data/EEG Eye State.arff")
    X = df.drop("eyeDetection", axis=1).to_numpy()
    y = df["eyeDetection"].astype(np.int).to_numpy()
    return X, y

In [77]:
get_eeg()

(array([[4329.23, 4009.23, 4289.23, ..., 4280.51, 4635.9 , 4393.85],
        [4324.62, 4004.62, 4293.85, ..., 4279.49, 4632.82, 4384.1 ],
        [4327.69, 4006.67, 4295.38, ..., 4282.05, 4628.72, 4389.23],
        ...,
        [4277.44, 3990.77, 4246.67, ..., 4257.95, 4591.79, 4339.49],
        [4284.62, 3991.79, 4251.28, ..., 4267.18, 4596.41, 4350.77],
        [4287.69, 3997.44, 4260.  , ..., 4274.36, 4597.95, 4350.77]]),
 array([0, 0, 0, ..., 1, 1, 1]))

## Loading CMAPSS

In [102]:
columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21"]

In [104]:
train=pd.read_csv("../data/CMAPSSData/train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("../data/CMAPSSData/test_FD001.txt",sep=" ",names=columns)
test_results=pd.read_csv("../data/CMAPSSData/RUL_FD001.txt",sep=" ",header=None)

In [106]:
train.head()

,id,cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,sensor22,sensor23
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


In [108]:
df_train = train
df_test = test

In [109]:
df_train['remaining_cycle'] = df_train.groupby(['id'])['cycle'].transform(max)-df_train['cycle']

In [125]:
df_grouped = df_train.groupby(["id"])
X, y = [], []
for eq_id, df_group in df_grouped:
    X.append([])
    y.append([])
    for row_index, row in df_group.iterrows():
        X[-1].append([row[c] for c in columns])
        y[-1].append(row["remaining_cycle"])
    print(len(X[-1]))
        
X, y = np.array(X), np.array(y)

192
287
179
189
269
188
259
150
201
222
240
170
163
180
207
209
276
195
158
234
195
202
168
147
230
199
156
165
163
194
234
191
200
195
181
158
170
194
128
188
216
196
207
192
158
256
214
231
215
198
213
213
195
257
193
275
137
147
231
172
185
180
174
283
153
202
313
199
362
137
208
213
213
166
229
210
154
231
199
185
240
214
293
267
188
278
178
213
217
154
135
341
155
258
283
336
202
156
185
200


/Users/e102949/.venvs/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


## Load individual household electric power consumption DS

In [130]:
df = pd.read_csv('../data/household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [134]:
df.to_numpy().shape

(2075259, 7)